In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import torchvision
torchvision.disable_beta_transforms_warning()

In [3]:
import os
import json

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# ollama
Settings.llm = Ollama(model="llama3.3", request_timeout=360.0, temperature=0)

In [5]:
format_title = lambda x: x.lower()

In [6]:
from llama_index.core.tools import FunctionTool
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent, FunctionCallingAgent, AgentRunner, FunctionCallingAgentWorker, StructuredPlannerAgent

In [18]:
from llama_parse import LlamaParse

In [33]:
parser = LlamaParse(
    result_type="markdown",
    premium_mode=False,  # "markdown" and "text" are available
    api_key = 'llx-EOjy0QMOscFhy9ZHigbX9YbfWO7X2nVdP3TdFjZ1QMXB7mWm'
)
file_extractor = {".pdf": parser}

In [43]:
with open('file_descriptions.json', 'r') as file:
    file_descriptions = json.load(file)

pdf_dir = 'data'

query_engine_tools = []
agents = {}

# Iterate through each PDF in the directory
for file_name in os.listdir(pdf_dir):
    if file_name.endswith(".pdf"):
        # Load the document
        file_path = os.path.join(pdf_dir, file_name)
        reader = SimpleDirectoryReader(input_files=[file_path], file_extractor=file_extractor)
        #reader = SimpleDirectoryReader(input_files=[file_path])
        documents = reader.load_data()
        description = file_descriptions[format_title(file_name[:-4])]
        
        # Create an index for the document
        vector_index = VectorStoreIndex.from_documents(documents)
        query_engine_tool = QueryEngineTool.from_defaults(
            vector_index.as_query_engine(
                similarity_top_k=5,
            ),
            name=f"{file_name[:-4]} query engine",
            description=description
        )
        
        query_engine_tools.append(query_engine_tool)

        agents[f"{format_title(file_name[:-4])}"] = FunctionCallingAgent.from_tools(
            [query_engine_tool],
            llm = Settings.llm,
            verbose=True,
            system_prompt=f"""\
            You are a specialized agent designed to answer questions about {format_title(file_name[:-4])} using the American \
            College of Radiology (ACR) appropriateness criteria. You have access to a tool that allows you to query a document \
            from the ACR regarding {format_title(file_name[:-4])}. This document {description.lower()} \
            In your response, provide as much relevant detail from the documents as possible. If a given scenario has a table with \
            ratings of the appropriate next steps, return each next step and its rating in a json format. Prioritize putting next steps \
            with high appropriateness ratings first. This may be indicated by a high number or a "usually appropriate" phrase. Do not \
            come up with ratings that are not explicitly given in the tools. \
            You must always use at least one of the tools provided when answering a question. Do NOT rely on prior knowledge.
            """
        )

Started parsing the file under job_id 1015c123-1edd-4882-883c-ac871ed55719
Started parsing the file under job_id 3b998941-3176-4761-83dd-62fe12d9ec2d
Started parsing the file under job_id 6c9d3754-9eb9-4483-8a8e-d7820092d5de
Started parsing the file under job_id 86efd569-f017-4a59-8b61-3af2ef4bfce5
Started parsing the file under job_id a27c2550-873f-4cce-a221-8667ff55f9ed
Started parsing the file under job_id afdcefde-1d9c-4daf-9701-b1da97d7107c
Started parsing the file under job_id 10b4d51c-5901-41e8-a6a4-a0645f6f1c37
Started parsing the file under job_id ee400e0e-3183-48d9-bd1f-e427212feb4e
Started parsing the file under job_id 214bab24-d1cc-436e-9f3d-5e9d5e892eb6
Started parsing the file under job_id 2fe9041d-ca16-4f19-9dca-a7dc77964a49
Started parsing the file under job_id 6e35c087-33e2-465a-83e3-378c2954b808
Started parsing the file under job_id 356336d6-33f8-4a7b-9224-7137e308a628
Started parsing the file under job_id 1e963773-78ed-48af-8fbe-3d344dfb89e9
Started parsing the file 

In [44]:
from llama_index.core.objects import SimpleToolNodeMapping, ObjectIndex

In [45]:
tools = []
for agent_name, agent in agents.items():
    tool = QueryEngineTool(
        query_engine = agent,
        metadata=ToolMetadata(
            name=f'tool_{agent_name}',
            description=f'This tool uses the {agent_name} agent to answer queries. {agent_name} has access to a document which {file_descriptions[agent_name].lower()}'
        )
    )
    tools.append(tool)

tool_mapping = SimpleToolNodeMapping.from_objects(tools)

In [46]:
fallback_tool = FunctionTool.from_defaults(
    lambda x: 'No relevant information contained in the American College of Radiology (ACR) appropriateness criteria',
    name="Fallback Tool",
    description="Handles queries with no relevance to interventional radiology  American College of Radiology (ACR) appropriateness criteria.\
    This tool can be called when no other tool can."
)
tools.append(fallback_tool)

In [47]:
obj_index = ObjectIndex.from_objects(
    tools + [fallback_tool],
    index_cls=VectorStoreIndex,
)

In [48]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [49]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=Settings.llm, 
    system_prompt=""" \
    You are a top-level agent designed to answer queries regarding planning, follow-up, and next steps for various interventional radiology \
    scenarios. Choose the most appropriate agent in the object index based on the user query. These agents have access to documents from \
    the American College of Radiology (ACR) appropriateness criteria. The agent will output a json if there is a matching scenario in its document. \
    You will receive queries regarding planning, follow-up, and next steps for various scenarios in interventional radiology and medicine more \
    generally. Use the agents and tools provided to you to best respond to these queries as if you were answering a question for a physician and \
    in a way that would be most useful for clincial practice. \
    For example, if a next step is regarded as "usually appropriate," you should respond as such. If you receive a json format text, interpret the text \
    and use the information in your response. Do not respond with json text. \
    If a response cannot be answered with the ACR Appropriateness criteria, state that the information is not contained within the ACR AC. \
    Prompt the user for additional information if it would be useful in narrowing down the next steps or recommendations. \
    Always use a provided agent to answer the question. Do NOT rely on prior knowledge.\
    """,
    verbose=True
)
top_agent = AgentRunner(agent_worker)

In [50]:
top_agent.chat('30 year old patient with fibroids who still wants to have kids. What should I do?')

Added user message to memory: 30 year old patient with fibroids who still wants to have kids. What should I do?
=== Calling Function ===
Calling function: tool_management of uterine fibroids with args: {"input": "30 year old patient with fibroids who still wants to have kids"}
> Running step 502f0a96-48ff-400d-9077-cd9b1015b4cc. Step input: 30 year old patient with fibroids who still wants to have kids
Added user message to memory: 30 year old patient with fibroids who still wants to have kids
=== Calling Function ===
Calling function: Management of Uterine Fibroids query engine with args: {"input": "30 year old patient with fibroids who still wants to have kids"}
=== Function Output ===
For a 30-year-old patient with uterine fibroids who wishes to preserve fertility, especially when considering the presence of concurrent adenomyosis, medical management is often considered a first-line approach due to its less invasive nature. This can include the use of progestin IUDs or combined oral

AgentChatResponse(response="For a 30-year-old patient with fibroids who still wants to have kids, the American College of Radiology (ACR) appropriateness criteria recommend medical management with progestin IUDs or combined oral contraceptives as usually appropriate. Laparoscopic or open myomectomy for bulk symptoms is also considered usually appropriate. Other options like MR-guided high-frequency focused ultrasound ablation and uterine artery embolization may be appropriate, while hysteroscopic myomectomy for submucosal fibroids is usually appropriate. \n\nCan you provide more information about the patient's specific symptoms and medical history? This would help in narrowing down the most suitable next steps.", sources=[ToolOutput(content='In this scenario, the American College of Radiology (ACR) appropriateness criteria would likely recommend the following next steps:\n\n```\n{\n  "next_steps": [\n    {\n      "step": "Medical management with progestin IUDs or combined oral contrace

In [51]:
top_agent.chat('The patient has heavy bleeding and submucosal fibroids')

Added user message to memory: The patient has heavy bleeding and submucosal fibroids
=== Calling Function ===
Calling function: tool_management of uterine fibroids with args: {"input": "heavy bleeding and submucosal fibroids"}
> Running step 33ebee6a-dd4a-4233-893b-0dfe01d56a31. Step input: heavy bleeding and submucosal fibroids
Added user message to memory: heavy bleeding and submucosal fibroids
=== Calling Function ===
Calling function: Management of Uterine Fibroids query engine with args: {"input": "heavy bleeding and submucosal fibroids"}
=== Function Output ===
For postmenopausal patients experiencing heavy bleeding and diagnosed with submucosal fibroids, particularly those with negative endometrial biopsy results, it's crucial to first rule out underlying causes such as uterine sarcoma due to the increased risk in this age group. Given the absence of relevant literature on medical management for this specific condition, uterine artery embolization (UAE) could be considered as a 

AgentChatResponse(response="For a patient with heavy bleeding and submucosal fibroids, uterine artery embolization (UAE) is usually an appropriate treatment option. Myomectomy is also usually appropriate, especially if the patient wishes to preserve fertility. Magnetic Resonance-guided Focused Ultrasound (MRgFUS) may be appropriate in certain cases. Hysterectomy could be considered for severe symptoms in patients who have completed childbearing.\n\nTo further guide management, can you provide more details about the patient's medical history, current symptoms, and any previous treatments or interventions she has undergone for her fibroids? Additionally, what are her preferences regarding future fertility preservation?", sources=[ToolOutput(content='Given the scenario of heavy bleeding and submucosal fibroids, the American College of Radiology (ACR) Appropriateness Criteria suggest the following management options:\n\n1. **Uterine Artery Embolization (UAE)**: This procedure is often reco

In [84]:
response = top_agent.query(
    'What kind of catheter should I use for central line? '
    'On a separate note, I also want to know if my patient with a DVT needs a stent. '
    'Also, do you know why blood is red? '
)

Added user message to memory: What kind of catheter should I use for central line? On a separate note, I also want to know if my patient with a DVT needs a stent. Also, do you know why blood is red? 
=== Calling Function ===
Calling function: tool_central venous access device and site selection with args: {"input": "central line catheter selection"}
> Running step 9b9f6374-9ec3-46e8-9268-bb03884805c8. Step input: central line catheter selection
Added user message to memory: central line catheter selection
=== Calling Function ===
Calling function: Central Venous Access Device and Site Selection query engine with args: {"input": "central line catheter selection"}
=== Function Output ===
For central line catheter selection, several options are considered based on patient needs and clinical scenarios. 

1. **Nontunneled Central Venous Catheter**: Usually appropriate for acutely ill patients requiring infusion of an irritant medication, hemodynamic monitoring, or frequent blood draws for 2

In [40]:
task = agent.create_task(
    'What kind of catheter should I use for central line? '
    'On a separate note, I also want to know if my patient with a DVT needs a stent. '
    'Also, do you know why blood is red? '
)

In [43]:
response = agent.chat(
    'What kind of catheter should I use for central line? '
    'On a separate note, I also want to know if my patient with a DVT needs a stent. '
    'Also, do you know why blood is red? '
)

Added user message to memory: What kind of catheter should I use for central line? On a separate note, I also want to know if my patient with a DVT needs a stent. Also, do you know why blood is red? 


KeyboardInterrupt: 

In [ ]:
agent = FunctionCallingAgent.from_tools(
    [all_pdfs_tool, fallback_tool],
    llm=Settings.llm,
    system_prompt='You are a specialized agent designed to answer queries about the planning and follow up \
    for topics related to interventional radiology based on the American College of Radiology (ACR) appropriateness criteria. \
    Answer using only information from one or more of the provided tools. \
    Do not answer from prior knowledge under any circumstances. \
    If no appropriate tool exists or if the tools do not provide sufficient context, do not answer the query. Instead, \
    indicate that the necessary information is not available in the ACR appropriateness criteria.',
    verbose=True
)

In [ ]:
agent = FunctionCallingAgent.from_tools(
    query_engine_tools + [fallback_tool],
    llm=Settings.llm,
    system_prompt='You are a specialized agent designed to answer queries about the planning and follow up \
    for topics related to interventional radiology based on the American College of Radiology (ACR) appropriateness criteria.\
    Given clinical vignettes, you are able to tell me the most appropriate next step and why. \
    If you tell me the imaging, please be as specific as possible, including utilization of contrast or location of the imaging.\
    The PDFs you are searching have a clinical vignette, followed by a table telling you the level of \
    appropriateness of the intervention/imaging. What you need to pay attention to are the ones that say Usually Appropriate. \
    If you tell me multiple potential options or next steps, tell me which are more appropriate. If one or more options is "Usually Appropriate",\
    explicitly state so. Answer using only information from one or more of the provided tools.\
    Do not answer from prior knowledge under any circumstances. If no appropriate tool exists or if the tools do not provide sufficient\
    context, do not answer the query. Instead, indicate that the necessary information is not available in the ACR\
    appropriateness criteria. Cite your sources. Once you have an answer, stop',
    verbose=True
)

In [ ]:
response = agent.chat(
    'What kind of catheter should I use for central line? '
    'On a separate note, I also want to know if my patient with a DVT needs a stent. '
    'Also, do you know why blood is red? '
)

In [85]:
response = top_agent.chat(
    '30 year old woman with fibroids who still desires pregnancy but is having trouble with fertility. What is the best next treatment plan?'
)

Added user message to memory: 30 year old woman with fibroids who still desires pregnancy but is having trouble with fertility. What is the best next treatment plan?
=== Calling Function ===
Calling function: tool_management of uterine fibroids with args: {"input": "fibroids and fertility issues in a 30-year-old woman"}
> Running step bffec16e-f97e-4423-8fb9-c3753ee10903. Step input: fibroids and fertility issues in a 30-year-old woman
Added user message to memory: fibroids and fertility issues in a 30-year-old woman
=== Calling Function ===
Calling function: Management of Uterine Fibroids query engine with args: {"input": "fibroids and fertility issues in a 30-year-old woman"}
=== Function Output ===
For a 30-year-old woman experiencing fibroids and fertility issues, several treatment options are available. Laparoscopic or open myomectomy is often considered appropriate for initial therapy, as it can help improve fertility outcomes. Medical management, such as ulipristal acetate, may 

In [ ]:
response = agent.chat(
    'Postmenopausal patient with uterine fibroids, symptomatic with heavy uterine bleeding or \
    bulk symptoms (eg, pressure, pain, fullness, bladder, or bowel symptoms). Negative \
    endometrial biopsy. Next step.'
)

In [ ]:
from llama_index.core import PromptTemplate

In [ ]:
agent = ReActAgent.from_tools(
    query_engine_tools + [fallback_tool],
    llm=Settings.llm,
    verbose=True
)

In [ ]:
agent.update_prompts({"agent_worker:system_prompt": react_system_prompt})

In [ ]:
response = agent.chat(
    'What kind of catheter should I use for central line? '
    'On a separate note, I also want to know if my patient with a DVT needs a stent. '
    'Also, do you know why blood is red? '
)

### Thoughts
- Can use as a training tool.
- Can be non-IR facing
- Can be IR facing

### Division of labor from here on out:
- one person starts user interface stuff
- one person makes the actual thing better (look into cohere api. josh said it was free)

### Other Thoughts
- I think this may be kinda shitty if we only include ACR AC. Should we think about incorporating other sources? If scaling up significantly, may need a way to filter tool selection. I think I saw how to do this somewhere.
- What is our big-picture goal with this? Could be good for pitch. e.g., who's target market
- Are there any benchmarks for this we can try? Could be nice to do a ChatGPT vs. our thing or something

### Other to dos
- query user for more info
- maintain history
- take in vignettes and also answer questions
- Cite sources/document used
- Gradio
- Decide on overall agentic rag pipeline/what works best in terms of which agents/how many

big picture todos:
1. performance
2. interface